In [4]:
import pandas as pd
import numpy as np
from IPython.display import display

import os
import json
import folium as fo

In [5]:
df = pd.read_csv('./unemployment_swiss_foreigner.csv', encoding='utf-8')

In [6]:
canton_list = [['Zurich', 'Berne', 'Lucerne', 'Uri', 'Schwyz', 'Obwald', 'Nidwald',
       'Glaris', 'Zoug', 'Fribourg', 'Soleure', 'Bâle-Ville', 'Bâle-Campagne',
       'Schaffhouse', 'Appenzell Rhodes-Extérieures',
       'Appenzell Rhodes-Intérieures', 'St-Gall', 'Grisons', 'Argovie',
       'Thurgovie', 'Tessin', 'Vaud', 'Valais', 'Neuchâtel', 'Genève',
       'Jura'],['ZH','BE','LU', 'UR', 'SZ', 'OW', 'NW', 'GL', 'ZG', 'FR',
               'SO', 'BS', 'BL', 'SH', 'AR', 'AI', 'SG', 'GR', 'AG', 'TG',
                'TI', 'VD', 'VS', 'NE', 'GE', 'JU']]

canton_dict = {canton_list[0][x] : canton_list[1][x] for x in range(len(canton_list[1]))}

In [7]:
unemployment = df[['Unnamed: 0', 'Nationalité', 'Total']]
unemployment.columns = ['Canton', 'Nationalité', 'Total']

unemployment = unemployment.drop(unemployment.index[0])

unemployment['short_name'] = unemployment.Canton.map(lambda x: canton_dict[x] if x in canton_dict else x)

unemployment = unemployment.apply(pd.to_numeric, errors='ignore')
unemployment.head()

,Canton,Nationalité,Total,short_name
1,Zurich,Etrangers,5.3,ZH
2,Zurich,Suisses,2.4,ZH
3,Berne,Etrangers,5.7,BE
4,Berne,Suisses,1.8,BE
5,Lucerne,Etrangers,4.0,LU


In [8]:
un_diff = pd.DataFrame()
un_diff['short_name'] = unemployment.short_name
un_diff['difference'] = unemployment.groupby(['Canton']).diff(-1)
un_diff = un_diff.dropna()

un_swiss = unemployment.loc[unemployment.Nationalité == 'Suisses']
un_foreigners = unemployment.loc[unemployment.Nationalité == 'Etrangers']
un_diff

,short_name,difference
1,ZH,2.9
3,BE,3.9
5,LU,2.8
7,UR,3.4
9,SZ,2.4
11,OW,1.9
13,NW,2.4
15,GL,2.6
17,ZG,2.2
19,FR,3.2


In [23]:
from branca.colormap import linear

colormap = linear.RdYlBu.scale(
    un_diff.difference.min(),
    un_diff.difference.max())

unemployment_dict = un_diff.set_index('short_name')['difference']

geo_path = r'topojson/ch-cantons.topojson.json'
geo_json_data = json.load(open(geo_path))

swiss_coord = [46.8182, 8.22]

ch_map = fo.Map(location=swiss_coord,
                tiles='cartodbpositron',
                zoom_start=8)

fo.TopoJson(
    geo_json_data, 
    'objects.cantons',
#     name='topofdsgfdgfdg',
    style_function = lambda feature: {
        'fillColor': colormap(unemployment_dict[feature['id']]),
        'color': 'black',
        'weight': 1,
        'dashArray': '5, 5',
        'fillOpacity': 0.7
    }
).add_to(ch_map)

ch_map
unemployment_dict
# colormap(2)
# geo_json_data

short_name
ZH    2.9
BE    3.9
LU    2.8
UR    3.4
SZ    2.4
OW    1.9
NW    2.4
GL    2.6
ZG    2.2
FR    3.2
SO    3.4
BS    2.1
BL    2.7
SH    3.2
AR    2.2
AI    2.3
SG    2.9
GR    2.7
AG    3.7
TG    2.7
TI    2.3
VD    2.6
VS    4.4
NE    3.8
GE    1.1
JU    5.8
Name: difference, dtype: float64

In [10]:
#geo_path = r'topojson/cantons_suisse_cantons.geojson'
geo_path = r'topojson/ch-cantons.topojson.json'
geo_json_data = json.load(open(geo_path))


ch_map = fo.Map(location=swiss_coord,
                tiles='cartodbpositron',
                zoom_start=8)

ch_map.choropleth(geo_data=geo_json_data, data=un_diff,
                  columns=['short_name', 'difference'],
                  key_on='feature.id',
                  fill_color='OrRd', fill_opacity=0.9, line_opacity=0.8,
                  legend_name = "rate difference between unemployed foreigners and swiss in (%)",
                  topojson = 'objects.cantons')
ch_map